<a href="https://colab.research.google.com/github/probabilityhill/gpt2-conditional-gen-en/blob/main/conditional_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

参考：https://towardsdatascience.com/conditional-text-generation-by-fine-tuning-gpt-2-11c1a9fc639d

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install git+https://github.com/huggingface/transformers  # ソースコードから直接transformersをインストール
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-kew6gwy9
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-kew6gwy9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.6 MB 8.3 MB/s 
     |████████████████████████████████| 101 kB 12.5 MB/s 
     |████████████████████████████████| 596 kB 60.2 MB/s 
  Created wheel for transformers: filename=transformers-4.21.0.dev0-py3-none-any.whl size=4594355 sha256=6af4ec33ea157ca1a100ea22cce2ada00960eac01d69b44231deaf9c14ab184c
  Stored in directory: /tmp/pip-ephem-wheel-cache-9hpl4tho/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninst

In [3]:
!git clone https://github.com/huggingface/transformers

# run_clm.pyを使う
!ls ./transformers/examples/pytorch/language-modeling/
# README.md   run_clm_no_trainer.py  run_mlm_no_trainer.py  run_plm.py
# requirements.txt  run_clm.py       run_mlm.py

Cloning into 'transformers'...
remote: Enumerating objects: 100722, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 100722 (delta 77), reused 105 (delta 35), pack-reused 100541
Receiving objects: 100% (100722/100722), 94.24 MiB | 17.15 MiB/s, done.
Resolving deltas: 100% (74266/74266), done.
README.md	  run_clm_no_trainer.py  run_mlm_no_trainer.py	run_plm.py
requirements.txt  run_clm.py		 run_mlm.py


In [4]:
MODEL = "gpt2"
OUTPUT_DIR = "drive/MyDrive/colab/output_gpt2/"
DATA_PATH = "drive/MyDrive/colab/data/rocstories.txt"
DATA_JOINED_PATH = "drive/MyDrive/colab/data/data_joined.txt"
SPECIAL_TOKENS = { "bos_token": "<|BOS|>",
                   "eos_token": "<|EOS|>",
                   "unk_token": "<|UNK|>",                    
                   "pad_token": "<|PAD|>",
                   "sep_token": "<|SEP|>"}

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
# AutoModelForPreTraining
tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer.add_special_tokens(SPECIAL_TOKENS)
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading

config = AutoConfig.from_pretrained(MODEL,
                                    bos_token_id=tokenizer.bos_token_id,
                                    eos_token_id=tokenizer.eos_token_id,
                                    sep_token_id=tokenizer.sep_token_id,
                                    pad_token_id=tokenizer.pad_token_id,
                                    output_hidden_states=False)
model = AutoModelForCausalLM.from_pretrained(MODEL, config=config)
#Special tokens added, model needs to be resized accordingly
model.resize_token_embeddings(len(tokenizer))

Embedding(50262, 768)

In [6]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

data_list = []  # 文の配列
sent = ""  # 整形済みの文＋改行
selected_list = []  # 抽出済みのキーワード
count = 0  # 使用する文数

with open(DATA_PATH, 'r', encoding="shift-jis") as f:
  data_list = f.readlines()

with open(DATA_JOINED_PATH, 'w', encoding="utf-8") as f:
  for data in data_list:
    data = data.lower()  # 全て小文字に
    morph = nltk.word_tokenize(data)  # 分かち書き
    pos = nltk.pos_tag(morph)  # 品詞取得
    keyword = ""

    # 名詞（なければ形容詞）を抽出
    for tag in ["NN", "NNS", "NNP", "NNPS", "JJ", "JJR", "JJS"]:
      keyword_list =  [x for x,y in pos if y == tag]  # 品詞がtagの単語の配列
      if keyword_list != []:
        for kw in keyword_list:
          # キーワードが重複しないように
          if kw not in selected_list:
            keyword = kw
            selected_list.append(keyword)
            break
        else:
          continue
        break
    
    # キーワードが抽出できなかったら飛ばす
    if keyword == "":
      continue
    
    sent = SPECIAL_TOKENS['bos_token'] + keyword + SPECIAL_TOKENS['sep_token'] + data.replace("\n", "") + SPECIAL_TOKENS['eos_token']
    f.write(sent + "\n")
    count += 1

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [37]:
print(sent)
print(f"文数：{count}")

<|BOS|>console<|SEP|>i wanted to buy a video game console.<|EOS|>
文数：3729


In [38]:
!python ./transformers/examples/pytorch/language-modeling/run_clm.py \
    --model_name_or_path=gpt2 \
    --train_file=drive/MyDrive/colab/data/data_joined.txt \
    --validation_file=drive/MyDrive/colab/data/data_joined.txt \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --do_train \
    --do_eval \
    --output_dir=drive/MyDrive/colab/output_gpt2/ \
    --num_train_epochs=5 \
    --save_total_limit=3 \
    --save_steps=10000 \
    --overwrite_output_dir


07/14/2022 09:27:38 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
07/14/2022 09:27:38 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_

In [39]:
model = AutoModelForCausalLM.from_pretrained(OUTPUT_DIR)

In [40]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer.do_lower_case = True
model.to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [41]:
def conditional_generation(keyword, num_gen=20):
    input_text = SPECIAL_TOKENS['bos_token']+keyword+SPECIAL_TOKENS['sep_token']
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    out = model.generate(input_ids, do_sample=True, top_p=0.95, top_k=40, num_return_sequences=num_gen, max_length=50, bad_words_ids=[[1], [3]])
    print("keyword: "+keyword)
    print("generated:")
    for sent in tokenizer.batch_decode(out):
      sent = sent.split(SPECIAL_TOKENS['sep_token'])[1].split(SPECIAL_TOKENS['eos_token'])[0]
      print(sent)

In [42]:
conditional_generation("happy")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


keyword: happy
generated:
the happy were her roommates and a lot of the other happy families.
she was happy for the first time in years.
the two of them enjoyed eating together.
jason's boyfriend asked what was happy and happy about his birthday today.
i had spent the last week trying to figure out why i was unhappy.
one day the family met up for dinner at the local park.
when they got home they found a happy happy little puppy.
he was happy to be home from work.
he was very happy to have been around her happily for so long.
he got a new bike and put it on happy!
i was glad she was happy because it took her a while to feel happy.
larry knew happy was his first job.
she felt happy and proud of the party.
my mom always told me that when the happy hours came he would go home happy.
neil was in love with a white cat.
jeff was pleased to find that his parents were happy.
mum said she had a happy happy birthday.
i'm happy to say i have a happy baby.
when he got home, his mom was unhappy with